# **Import Libraries**

In [ ]:
!apt-get install swig -y
!pip install Cython numpy
!curl https://raw.githubusercontent.com/automl/smac3/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install smac
!pip install wget
!pip install plotly
!pip install -Uqq ipdb

import ipdb

import logging
import time

from ConfigSpace.conditions import InCondition
from ConfigSpace.hyperparameters import CategoricalHyperparameter, \
    UniformFloatHyperparameter, UniformIntegerHyperparameter
from smac.configspace import ConfigurationSpace
from smac.scenario.scenario import Scenario
from smac.facade.smac_hpo_facade import SMAC4HPO
from smac.initial_design.default_configuration_design import DefaultConfiguration

import torch
import wget
import zipfile 
import os
import pandas as pd
import numpy as np
from statistics import mean
from IPython.display import clear_output
from random import randrange

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
from IPython.display import clear_output
import random

from abc import ABC
import scipy as sc
import pickle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,474 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpackin

In [ ]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")
device

device(type='cuda')

# **Global Variables**

In [ ]:
# Settings for experiments
stop_chunk = None
epochs = 20

K_range = np.arange(2,200+2)
lr_range = np.linspace(1e-5, 1e-2, 100)

starting_index=0


In [ ]:
ML_1M = 'ml_1m'
ML_25M = 'ml_25m'

dataset = ML_1M

# **Data Sources**

### **Data Loader**

In [ ]:
class DataLoader:
    def __init__(self,
                 url='http://files.grouplens.org/datasets/movielens/ml-1m.zip',
                 archive_name='dataset_ml_1m.zip',
                 filename='/ml-1m/ratings.dat', 
                 sep='::',
                 skiprows=0, 
                 force_download=False,
                 drift=False,
                 batch_size=100):
      
        if not os.path.isfile(archive_name) or force_download:
            wget.download(url, archive_name)
            files = zipfile.ZipFile(archive_name, 'r')
            files.extractall('./data')
            files.close()

        self.df=pd.read_csv('./data'+filename,sep=sep, 
                              names=['User','Item','Rating','Timestamp'],
                              skiprows=skiprows)
        
        self.df=self.df.sort_values('Timestamp')

        self.M=self.df['User'].max()
        self.N=self.df['Item'].max()

        self.df['User'] = self.df['User'] - 1
        self.df['Item'] = self.df['Item'] - 1

        self.drift = drift
        self.batch_size = batch_size
        self.warning = True

        print('Number of Users:', self.M)
        print('Number of Items:', self.N)

        self.number_of_samples = len(self.df.index)
        print('Samples:',self.number_of_samples)
        
        self.batches = int(math.ceil(float(self.number_of_samples)/float(self.batch_size)))
        
    def iter(self):
        idx_list = np.arange(len(self.df.index))
        
        # np.random.shuffle(idx_list)
        size = idx_list.shape[0]
        

        for start_idx in range(0, size, self.batch_size):

            end_idx = min(start_idx + self.batch_size, size)
            
            data= self.df.iloc[idx_list[start_idx:end_idx],:]
            data_np=data.to_numpy()

            rating = data_np[:,2]

            if self.drift and start_idx>size/2 :
              rating = 6-rating
              if self.warning:
                print('Starting Drifting: all the ratings will be inverted!')
                self.warning=False

            yield data_np[:,0], data_np[:,1], rating  # users, items, ratings

In [ ]:
testDataLoader=DataLoader()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Number of Users: 6040
Number of Items: 3952
Samples: 1000209


In [ ]:
c=0

for u,i,r in testDataLoader.iter():
  # extract the current mini-batch =============================================
  print(u)
  print(i)
  print(r)

  if c==2:
    break
  
  c+=1

[6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039
 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039
 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039
 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039 6039
 6039 6039 6039 6039 6039 6039 6038 6038 6039 6038 6039 6039 6038 6038
 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038
 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038 6038
 6038 6038]
[ 857 2383  592 1960 2018 1418  212 3110  572 3504 1733 2502  918  911
  526 1251  648  317 3288  758 2857  607 2395 2027  325 1648   16 2761
   33  245 2691 1616  299 1391 1110  149  561  548 1553  264  447  865
 1536 2323 1357  234  246  445 1703 1093   49   44  161  347  507 1088
  588   57 1693 2579 1833 2390  110  281  289 2066 1640  356 1229  929
  946 3087 3021 3132 1293 2803 3420 1268 1243  954 1275 2621 2790 2299
 1027 2395 1196 3547 2862 1222  950 1210  932 3071 1065  906  934

### **Data Generator**

In [ ]:
class DataGenerator(ABC):

  def get_frequencies(α, K, N, debug=False):
    x = (K-1)*np.random.rand(K)+1
    x.sort()
    f = N*np.power(x,-α)
    return f.astype(int)

  def __init__(self, N, M, K, 
               maxRating=5, 
               batches=100, 
               batch_size=256, 
               α=1.1, 
               p=1, 
               p_d=1,
               p_r=1,
               p_a=1,
               gaussian_sampling=False,
               variance=1,
               debug=False, 
               save_dataset=False):
    self.N = N
    self.M = M
    self.K = K
    self.maxRating = maxRating - 1
    self.batches = batches
    self.batch_size = batch_size
    self.α = α
    self.p = p
    self.p_d = p_d
    self.p_r = p_r
    self.p_a = p_a
    self.gaussian_sampling = gaussian_sampling
    self.variance = variance
    self.deterministic = False
    self.debug = debug
    self.timestamp = 0 
    self.batches_arr = None
    self.P = np.zeros((self.N,self.K))
    self.Q = np.zeros((self.M,self.K))
    self.__assign_features(np.arange(self.K))
    self.__update_ratings()
    
    self.save_dataset = save_dataset

  def set_deterministic(self, deterministic):
    self.deterministic = deterministic
    
    if self.deterministic:
      self.batches_arr = []

      for u,i,r in self.iter_():
          self.batches_arr.append([u,i,r])

      if self.save_dataset:
        d={
            'data' : self.batches_arr,
            'N': self.N,
            'M': self.M,
            'batches': self.batches,
            'batch_size': self.batch_size
        }
        with open('synthetic_data.dat', 'wb') as f:
          pickle.dump(d, f)


  def reset_timestamp(self):
    self.timestamp = 0

  def set_timestamp(self, timestamp):
    self.timestamp = timestamp

  def __update_ratings(self):
    self.R = np.round(self.P.dot(self.Q.T)+1)

  def plot_ratings(self, rating=None):
    if rating is None:
      R_ = (self.R>3).astype(float)
    else:
      R_ = (self.R==rating).astype(float)
    freqs_users=np.sum(R_,axis=1)
    freqs_items=np.sum(R_,axis=0)
    print('freqs_users.shape',freqs_users.shape)
    print('freqs_items.shape',freqs_items.shape)
    fig, axs = plt.subplots(2, 2, figsize=(10,10))
    axs[0, 0].plot(sorted(freqs_users,reverse=True),'r')
    axs[0, 1].loglog(range(1,self.N+1),sorted(freqs_users,reverse=True))
    axs[1, 0].plot(sorted(freqs_items,reverse=True),'r')
    axs[1, 1].loglog(range(1,self.M+1),sorted(freqs_items,reverse=True))
    axs[0, 0].set_title('Users')
    axs[0, 1].set_title('Users (log)')
    axs[1, 0].set_title('Items')
    axs[1, 1].set_title('Items (log)')
    for i in range(2):
      for j in range(2):
        axs[i, j].grid()

    plt.show()

  def get_normalized_degrees(self, M, axis=1, l=None):

    # axis=1 : degrees of users/items
    # axis=0 : degrees of features

    d=np.sum(M,axis=axis)

    if np.sum(d)==0:
      d=np.ones(M.shape[1-axis])

    if l is not None:
      d[l]=0

    # normalization
    d=d/np.sum(d)

    return d

  def get_connected_feature(self, M, obj):
    p=(M[obj,:]>0).astype(float)
    s=np.sum(p)
    if s==0:
      # no feature is connected with the object (user or item) obj
      return None

    p=p/np.sum(p)
    connected_feature = np.random.choice(self.K, p=p)

    return connected_feature


  def __assign_features(self, features, features_to_unassign=None, enable_rewiring=False):

    P_freqs = DataGenerator.get_frequencies(self.α, self.K, self.N)
    Q_freqs = DataGenerator.get_frequencies(self.α, self.K, self.M)
    
    for feature in features:
      i = np.random.randint(self.K) 

      P_freq = max(1,P_freqs[i]) # number of users to associate to this feature
      Q_freq = max(1,Q_freqs[i]) # number of items to associate to this feature

      uniform_sampling=np.random.choice(2, 1, p=[self.p, 1-self.p])[0]==0

      if self.debug:
        print('uniform_sampling:',uniform_sampling)

      if uniform_sampling:
        users_to_update = random.sample(range(self.N), P_freq)
        items_to_update = random.sample(range(self.M), Q_freq)
      else:
        users_normalized_degrees = self.get_normalized_degrees(self.P)
        items_normalized_degrees = self.get_normalized_degrees(self.Q)

        if self.debug:
          print('self.N',self.N)
          print('users_normalized_degrees',users_normalized_degrees.shape)

        users_to_update = np.random.choice(self.N, P_freq, p=users_normalized_degrees) 
        items_to_update = np.random.choice(self.M, Q_freq, p=items_normalized_degrees) 

      for j in np.arange(P_freq): 
        user = users_to_update[j]
        
        if enable_rewiring:
          rewire=np.random.choice(2, 1, p=[self.p_a, 1-self.p_a])[0]==1
          if rewire:
            # rewire
            # if it is possible remove an old edge randomly selected
            feature_to_disconnect = self.get_connected_feature(self.P, user)
            if feature_to_disconnect is not None:
              self.P[user,feature_to_disconnect] = 0
              if self.debug:
                print('Rewiring (user,feature) edge. Removing the edge (',user,',',feature_to_disconnect,')',sep='')
          
        # create the new edge
        if self.debug:
          print('Adding the (user, feature) edge (',user,',',feature,')',sep='')
        self.P[user,feature] = 1

        if features_to_unassign is not None:
          self.P[user,features_to_unassign] = 0

      # self.P[users_to_update,feature] = 1
      
      for j in np.arange(Q_freq): 
        item = items_to_update[j]

        if self.debug and j==10:
          print('Old Q[',item,']: ',self.Q[item],sep='')
        
        if enable_rewiring:
          rewire=np.random.choice(2, 1, p=[self.p_a, 1-self.p_a])[0]==1
          if rewire:
            # rewire
            # if it is possible remove an old edge randomly selected
            feature_to_disconnect = self.get_connected_feature(self.Q, item)
            if feature_to_disconnect is not None:
              self.Q[item,feature_to_disconnect] = 0
              if self.debug:
                print('Rewiring (item,feature) edge. Removing the edge (',item,',',feature_to_disconnect,')',sep='')
          
        # create the new edge
        if self.debug:
          print('Adding the (item, feature) edge (',item,',',feature,')',sep='')
        self.Q[item,feature] = 1

        if features_to_unassign is not None:
          self.Q[item,features_to_unassign] = 0

        # further ops are required for items
        row = (self.Q[item]>0).astype(float) # [0,1,0,1,0,0,0,0,...,1]
        c = np.sum(row)
        row = row * self.maxRating / c
        
        self.Q[item] = row

        if self.debug and j==10:
          print('New Q[',item,']: ',self.Q[item],sep='')

  def add_features(self, number_of_features, unassign_old_features=True):
    self.P = np.hstack((self.P, np.zeros((self.P.shape[0], number_of_features), dtype=self.P.dtype)))
    self.Q = np.hstack((self.Q, np.zeros((self.Q.shape[0], number_of_features), dtype=self.Q.dtype)))

    self.K=self.K+number_of_features

    if unassign_old_features:
      self.__assign_features(list(range(self.K-number_of_features,self.K)), list(range(self.K-number_of_features)), enable_rewiring=True)
    else:
      self.__assign_features(list(range(self.K-number_of_features,self.K)), None, enable_rewiring=True)
      
    self.__update_ratings()
  
  def add_users(self, number_of_users):
    self.P = np.vstack((self.P, np.zeros((number_of_users, self.P.shape[1]), dtype=self.P.dtype)))

    self.N = self.N + number_of_users

    freqs = DataGenerator.get_frequencies(self.α, self.N, self.K)

    for user in range(self.N-number_of_users, self.N):
      # creating a user
      i = np.random.randint(self.N) 

      # number of features to assign to the user
      freq = max(1,freqs[i])

      features_to_update = random.sample(range(self.K), freq)

      if self.debug:
        print('freqs:', freqs)
        print('features_to_update:', features_to_update)
      
      for feature in features_to_update: 
        self.P[user,feature] = 1

    self.__update_ratings()

  def add_items(self, number_of_items):
    self.Q = np.vstack((self.Q, np.zeros((number_of_items, self.Q.shape[1]), dtype=self.P.dtype)))

    self.M=self.M+number_of_items

    freqs = DataGenerator.get_frequencies(self.α, self.M, self.K)

    for item in range(self.M-number_of_users, self.M):
      # creating a item
      i = np.random.randint(self.M) 

      # number of features to assign to the item
      freq = max(1,freqs[i])

      features_to_update = random.sample(range(self.K), freq)

      if self.debug:
        print('freqs:', freqs)
        print('features_to_update:', features_to_update)
      
      for feature in features_to_update: 
        self.Q[item,feature] = 1/freq

    self.__update_ratings()

  def delete_rewire_edges(self, M, feature, features_to_remove):

    # Delete/Rewire (object,feature) edges, where object is a user or an item

    for obj in range(M.shape[0]):
      if M[obj,feature]>0:

        remove_edge=np.random.choice(2, 1, p=[self.p_d, 1-self.p_d])[0]==0
        if self.debug:
          if remove_edge:
            print('The edge (',obj,',',feature,') will be removed',sep='')
          else:
            print('The edge (',obj,',',feature,') will be rewired',sep='')
        if not remove_edge:
          # rewire!
          uniform_sampling=np.random.choice(2, 1, p=[self.p_r, 1-self.p_r])[0]==0

          if self.debug:
            # print('uniform_sampling (rewire):',uniform_sampling)
            pass

          if uniform_sampling:
            # uniform sampling
            if self.debug:
              print('Rewiring with uniform sampling')
            feature_to_update = random.choice(features)
          else:
            # prob. proportional to the degree of the feature
            features_normalized_degrees = self.get_normalized_degrees(M,axis=0, l=features_to_remove)
            feature_to_update = np.random.choice(self.K, p=features_normalized_degrees) 
            if self.debug:
              print('Rewiring with prob. proportional to the degree of the feature')
              print('features_normalized_degrees:',features_normalized_degrees)
              print('Feature selected:',feature_to_update)

          if self.debug:
              print('The edge (',obj,',',feature,') will be replace with the edge (',obj,',',feature_to_update,')',sep='')

          M[obj,feature_to_update]=1

  def remove_features(self, number_of_features):
    if number_of_features>self.K-1:
      return 

    # column to remove
    # i = self.K - min(np.random.zipf(self.α),self.K)
    # i = min(np.random.zipf(self.α),self.K)-1

    features_to_remove = random.sample(range(self.K), number_of_features)

    if self.debug:
      print('Features to remove:',features_to_remove)

    features = np.arange(self.K)

    if self.debug:
      print('Old list of features:',features)

    features=np.setdiff1d(features, features_to_remove)

    if self.debug:
      print('New list of features:',features)
    
    # See the paper for details
    for feature in features_to_remove:
      print('Feature to remove: ',feature,':',sep='')

      # Delete/Rewire (user,feature) edges
      if self.debug:
        print('Delete/Rewire (user, ',feature,') edges ==============================',sep='')
      self.delete_rewire_edges(self.P,feature,features_to_remove)

      # Delete/Rewire (item,feature) edges
      if self.debug:
        print('Delete/Rewire (item, ',feature,') edges ==============================',sep='')
      self.delete_rewire_edges(self.Q,feature,features_to_remove)

    self.P = self.P[:, features]
    self.Q = self.Q[:, features]

    self.Q=(self.Q>0).astype(float)
    c=self.Q.sum(axis=1)[:,None]
    c=np.maximum(c,1)
    self.Q=self.Q/c
    self.Q=self.Q*self.maxRating

    self.K -= number_of_features
    self.__update_ratings()

  def invert_preferences(self):
    self.P = 1 - self.P
    self.__update_ratings()

  def drift(self):
    pass

  def time_to_drift(self):
    return False

  def sample_ratings(self, batch_size):
    lu = list(range(self.N))
    li = list(range(self.M))

    users = random.choices(lu, k=batch_size)
    items = random.choices(li, k=batch_size)

    avgs = self.R[users,items]
    samples = None

    if self.gaussian_sampling:
      samples = np.random.normal(size=batch_size)
      ratings = np.clip(np.round(math.sqrt(self.variance)*samples + avgs),1,self.maxRating+1)
    else:
      ratings = avgs

    if self.debug:
      print('users:',users)
      print('items:',items)
      print('avgs:',avgs)
      print('samples:',samples)
      print('ratings:',ratings)

    # increase timestamp
    self.timestamp += 1

    return items, users, ratings

  def iter_(self):
    for i in range(self.batches):
      if self.time_to_drift():
        print('Drifting...')
        self.drift()
      yield self.sample_ratings(self.batch_size)  # users, items, ratings

  def iter(self):
    if self.deterministic:
      for u,i,r in self.batches_arr[0:len(self.batches_arr)]:
        yield u,i,r
    else:
      for i in range(self.batches):
        if self.time_to_drift():
          print('Drifting...')

          self.drift()
        yield self.sample_ratings(self.batch_size)  # users, items, ratings

In [ ]:
# A DataGenerator implementing a drift strategy
class DataGenerator_0(DataGenerator):

  def __init__(self, N, M, K, 
               maxRating=5, 
               batches=100, 
               batch_size=256, 
               T=10,
               α=1.1, 
               p=1, 
               p_d=1,
               p_r=1,
               p_a=1,
               gaussian_sampling=False,
               variance=1,
               debug=False, 
               save_dataset=False):
    super().__init__(N, M, K, 
               maxRating, 
               batches, 
               batch_size, 
               α, 
               p, 
               p_d,
               p_r,
               p_a,
               gaussian_sampling,
               variance,
               debug, 
               save_dataset)

    self.T = T

  def time_to_drift(self):
    if self.timestamp > 0 and self.timestamp % self.T==0:
      return True
    return False

  def drift(self):
    self.invert_preferences()

In [ ]:
class DataGenerator_1(DataGenerator):

  def __init__(self, N, M, K, 
               maxRating=5, 
               batches=100, 
               batch_size=256, 
               T=10,
               α=1.1, 
               p=1, 
               p_d=1,
               p_r=1,
               p_a=1,
               gaussian_sampling=False,
               variance=1,
               debug=False, 
               save_dataset=False,
               unassign_old_features=False):
    super().__init__(N, M, K, 
               maxRating, 
               batches, 
               batch_size, 
               α, 
               p, 
               p_d,
               p_r,
               p_a,
               gaussian_sampling,
               variance,
               debug, 
               save_dataset)

    self.T = T
    self.unassign_old_features = unassign_old_features
    self.H = K

  def time_to_drift(self):
    if self.timestamp > 0 and self.timestamp % self.T==0:
      return True
    return False

  def drift(self):
    self.add_features(self.H, self.unassign_old_features)

### **Synthetic Data Loader**

In [ ]:
class SyntheticDataLoader():

    def __init__(self,  url='https://www.dropbox.com/s/wta2a2uylzfjarc/synthetic_data.dat?dl=1', filename='synthetic_data.dat', force_download=False):
      
      if not os.path.isfile(filename) or force_download:
            wget.download(url, filename)

      with open (filename, 'rb') as f:
        d = pickle.load(f)
               
        self.batches_arr = d['data']
        self.N = d['N']
        self.M = d['M']
        self.batches = d['batches']
        self.batch_size = d['batch_size']

    def iter(self):
      for u,i,r in self.batches_arr[0:len(self.batches_arr)]:
        yield u,i,r


# **Plot**

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

def plot_models(simplex_models, aux_models, K_range, lr_range, plot_lines=True, show=True):
  models = {**simplex_models, **aux_models}

  x = [models[model].K for model in models]
  y = [models[model].lr for model in models]
  
  label = [model for model in models]
  loss = [models[model].loss for model in models]

  hovertext = ['['+model+'] Loss: ' + str(models[model].loss) for model in models]
  
  fig = go.Figure()

  fig.update_layout(
      autosize=False,
      width=500,
      height=500)
  
  fig.update_xaxes(range=[K_range[0]-K_range[0]*0.5,K_range[-1]+K_range[-1]*0.5])
  fig.update_yaxes(range=[lr_range[0]-lr_range[0]*0.5,lr_range[-1]+lr_range[-1]*0.5])

  x_simplex = [models['B'].K,models['G'].K,models['W'].K,models['B'].K]
  y_simplex = [models['B'].lr,models['G'].lr,models['W'].lr,models['B'].lr]
  
  x_aux = [models['B'].K, models['E'].K, models['G'].K, models['C'].K, models['B'].K, models['R'].K, models['G'].K]
  y_aux = [models['B'].lr,models['E'].lr,models['G'].lr,models['C'].lr,models['B'].lr,models['R'].lr,models['G'].lr]

  simplex_scatter = None
  aux_scatter = None

  if plot_lines:
      simplex_scatter = go.Scatter(x=x_simplex, y=y_simplex,
                      mode='lines',
                      opacity=0.8,
                      marker={'color': 'blue'},
                      showlegend=False)
      fig.add_trace(simplex_scatter)
      
      aux_scatter = go.Scatter(x=x_aux, y=y_aux,
                      mode='lines',
                      opacity=0.4,
                      marker={'color': 'gray'},
                      showlegend=False
                      )
      fig.add_trace(aux_scatter)
  
  scatter = go.Scatter(x=x, 
                       y=y,
                        marker={'color': loss, 'showscale': True},
                        mode='markers+text',
                        name='Model',
                        text=label,
                        showlegend=False,
                        textposition='top center',
                       hovertext=hovertext)
  fig.add_trace(scatter)
  
  if show:
    fig.show()

  return [scatter, simplex_scatter, aux_scatter]
  

# **Drift Detector - Page Hinkley**

In [ ]:
class PageHinkleyJG():

    def __init__(self, delta=0.005, threshold=50):
        self.change_u = None
        self.delta = delta
        self.threshold = threshold
        self.S = 0
        self.U = 0
        self.m = 0
        self.t = 1
        self.PH_U = []

    def reset(self):
        self.change_u = None
        self.S = 0
        self.U = 0
        self.m = 0
        self.t = 1

    def add_element(self, x):
        self.t += 1
        self.S += x
        self.U = ((self.t - 1) / self.t) * self.U + x - (self.S / self.t) - self.delta
        self.m = min(self.m, self.U)
        phu = self.U - self.m
        self.PH_U.append(phu)
        if phu >= self.threshold:
            self.change_u = True
            #self.reset()
        else:
            self.change_u = False
        return phu

    def detected_change(self):
        return self.change_u
        

# **Model - Matrix Factorization**

In [ ]:
from numpy.linalg import lstsq

def find_orth(O):
    rand_vec = np.random.rand(O.shape[0], 1)
    A = np.hstack((O, rand_vec))
    b = np.zeros(O.shape[1] + 1)
    b[-1] = 1
    v = lstsq(A.T, b)[0]
    return v

def find_all_orth(O, h):
  M = O.cpu().detach()
  shape=M.size()
  R = torch.rand(shape[0], h)
  A = torch.cat((M,R),dim=1)
  b_0 = torch.zeros((shape[1],h))
  b_1 = torch.eye(h)
  b = torch.cat((b_0, b_1),dim=0)
  v,_ = torch.lstsq(b,torch.transpose(A,0,1))
  return v.to(device)

class MatrixFactorization(torch.nn.Module):
    def __init__(self, M, N, K=20, lr=10e-3, model=None, optimization=True):
        super().__init__()
        self.loss = None
        self.M=M
        self.N=N
        self.K=K
        self.lr=lr
        if model is None:
          self.P = torch.nn.Embedding(M, K)
          self.Q = torch.nn.Embedding(N, K)
        else:
          self.loss = model.loss
          if self.K==model.K:
              P_weight = model.P.weight
              Q_weight = model.Q.weight
          elif self.K>model.K:
            if optimization:
              h = self.K - model.K
              P_weight = model.P.weight
              Q_weight = model.Q.weight
              P_orth_columns = find_all_orth(P_weight, h)
              Q_orth_columns = find_all_orth(Q_weight, h)
              P_weight = torch.cat((P_weight,P_orth_columns),dim=1)
              Q_weight = torch.cat((Q_weight,Q_orth_columns),dim=1)
            else:
              P_weight = torch.rand(M,K)
              Q_weight = torch.rand(N,K)
              P_weight[:,:model.K] = model.P.weight
              Q_weight[:,:model.K] = model.Q.weight
          else:
            if optimization:              
              P_weight = model.P.weight[:,self.get_highest_variance_columns(model.P.weight)]
              Q_weight = model.Q.weight[:,self.get_highest_variance_columns(model.Q.weight)]
            else:
              P_weight = model.P.weight[:,:self.K]
              Q_weight = model.Q.weight[:,:self.K]
          
          self.P=torch.nn.Embedding.from_pretrained(P_weight,freeze=False)
          self.Q=torch.nn.Embedding.from_pretrained(Q_weight,freeze=False)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    
    def get_highest_variance_columns(self, M):
        std = torch.std(M.data, dim=0)
        sorted_idx = torch.argsort(std,descending=True)
        highest_variance_columns = np.sort(sorted_idx[:self.K].cpu().detach().numpy())
        return highest_variance_columns

    def forward(self, users, items):
        return (self.P(users) * self.Q(items)).sum(axis=1)
    
    def __repr__(self):
        return 'K: '+str(self.K)+', lr: '+str(self.lr)+', '+'loss: '+str(self.loss)
    
    def __lt__(self, other):
        return self.loss < other.loss 
    
    def find_orth(O):
      rand_vec = np.random.rand(O.shape[0], 1)
      A = np.hstack((O, rand_vec))
      b = np.zeros(O.shape[1] + 1)
      b[-1] = 1
      v = lstsq(A.T, b)[0]
      return v

    def find_all_orth(O, h):
      M = O.cpu().detach()
      shape=M.size()
      R = torch.rand(shape[0], h)
      A = torch.cat((M,R),dim=1)
      b_0 = torch.zeros((shape[1],h))
      b_1 = torch.eye(h)
      b = torch.cat((b_0, b_1),dim=0)
      v,_ = torch.lstsq(b,torch.transpose(A,0,1))
      return v.to(device)

# **Evaluation Metrics**

In [ ]:
def RMSELoss(y_hat,y):
    return torch.sqrt(torch.mean((y_hat-y)**2))

loss_func = RMSELoss

# **Animation**

In [ ]:
import plotly.graph_objects as go

def animate(plots):
  # frames=[go.Frame(data=sl) for sl in plots[1:]]
  frames=[]
  j=0
  for s1 in plots[1:]:
    frame = {"data": s1, "name": str(j)}
    j+=1
    frames.append(frame)
  #print(frames)
    
  steps = []
  #print(frames)
  for i in range(len(frames)):
      slider_step = {"args": [
            [i],
            {"frame": {"duration": 300, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 300}}
        ],
            "label": i,
            "method": "animate"}
      # step = dict(
      #     method="update",
      #     args=[{"visible": [False] * len(frames)},
      #           {"title": "Slider switched to step: " + str(i)}],  # layout attribute
      # )
      # step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
      steps.append(slider_step)

  sliders = [dict(
      active=0,
      currentvalue={"prefix": "Plot: "},
      pad={"t": len(frames)},
      steps=steps
  )]
  fig = go.Figure(
      data=frames[0]['data'],
      layout=go.Layout(
          autosize=False,
          width=600,
          height=600,
          xaxis=dict(range=[K_range[0]-25,K_range[-1]+25], autorange=False),
          yaxis=dict(range=[lr_range[0]-0.005,lr_range[-1]+0.005], autorange=False),
          xaxis_title="Hyperparameter: Embedding Size",
          yaxis_title="Hyperparameter: Learning Rate",
          title="Optimization",
          updatemenus=[dict(
              type="buttons",
              x=0.0,
              xanchor="left",
              y=1.1,
              direction= "left",
              yanchor="top",
              buttons=[dict(label="Play",
                            method="animate",
                            args=[None,{"frame": {"duration": 500, "redraw": False},
                                  "fromcurrent": True, "transition": {"duration": 300,
                                                                      "easing": "quadratic-in-out"}}]),
                      dict(label="Pause",
                            method="animate",
                            args=[[None],{"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])])],
          sliders = sliders),
      frames=frames
  )
  fig.show()

# **Nelder-Mead**

In [ ]:
  def process_streaming(dataSource, pht_threshold, batch_size, chunk_size, window_size, debug=True):
    start_time_runtime = time.time()
    start_time_convergence = time.time()
    def build_aux_models(simplex_models):
      aux_models = {'M': None, 
                  'R': None, 
                  'E': None, 
                  'S': None, 
                  'C': None}
                  
      B=simplex_models['B']
      G=simplex_models['G']
      W=simplex_models['W']

      template_model = B # in Java code is W

      # Create M
      K = check_range(int(round((B.K + G.K)/2,0)),K_range)
      lr= check_range((B.lr + G.lr)/2,lr_range)
      M = MatrixFactorization(B.M,B.N,K,lr,template_model)

      # Create R
      K = check_range(int(round(2*M.K - W.K,0)),K_range) 
      lr = check_range(2*M.lr - W.lr,lr_range)
      R = MatrixFactorization(B.M,B.N,K,lr,template_model)

      # Create E
      K = check_range(int(round(2*R.K - M.K,0)),K_range) 
      lr = check_range(2*R.lr - M.lr,lr_range)
      E = MatrixFactorization(B.M,B.N,K,lr,template_model)

      # Create S
      K = check_range(int(round((B.K + W.K)/2,0)),K_range)
      lr= check_range((B.lr + W.lr)/2,lr_range)
      S = MatrixFactorization(B.M,B.N,K,lr,template_model)

      # Create C1
      K = check_range(int(round((M.K + W.K)/2,0)),K_range)
      lr= check_range((M.lr + W.lr)/2,lr_range)
      C = MatrixFactorization(B.M,B.N,K,lr,template_model)

      aux_models['M'] = M
      aux_models['R'] = R
      aux_models['E'] = E
      aux_models['S'] = S
      aux_models['C'] = C

      return aux_models

    def check_range(x, range):
      if x<range[0]:
        return range[0]
      if x>range[-1]:
        return range[-1]
      return x

    def train_models(models, users, items, ratings, epochs):
      for model_name in models:
        for epoch in range(epochs):
          model = models[model_name]
          model.optimizer.zero_grad()      
          prediction = model(users, items)
          loss = loss_func(prediction, ratings)
          model.loss = loss.item()
          loss.backward()
          model.optimizer.step()
          
    total_loss = 0
    number_of_chunks = 0
    plots = []
    average_loss_values = []
    batch_loss_values = []
    convergencesphere=2.001
    number_of_batches_in_chunk=0
    number_of_batches=0
    window_loss=[]
    window=[]
    plot_training = False
    convergence=False
    avg_time_simplex=0
    avg_time_auxmodels=0
    counter_simplex=0
    counter_auxmodels=0
    start_time_runtime = time.time()
    start_time_convergence = time.time()
    count_examples_runtime = 0 
    count_examples_convergence = 0 
    initialtimes=[]
    endtimes=[]
    PHm_T = []
    pht=PageHinkleyJG(threshold=pht_threshold)
    initialtimes.append(time.time())

    start = True
      
    for u,i,r in dataSource.iter():

      # extract the current mini-batch =========================================
      users = torch.LongTensor(u).to(device)
      items = torch.LongTensor(i).to(device)
      ratings = torch.FloatTensor(r).to(device)
      # ========================================================================

      number_of_batches +=1
      number_of_batches_in_chunk +=1

      if start:
        # start
        simplex_models = {'B': None, 
                          'G': None, 
                          'W': None}

        gamma=4
        offset=(1-1/gamma)/2

        # initialize simplex models ============================================
        for model_name in simplex_models:
          K = K_range[int(K_range.size*offset)+randrange(int(K_range.size/gamma))]
          lr = round(lr_range[int(lr_range.size*offset)+randrange(int(lr_range.size/gamma))],6)
          print('Initializing model', model_name)
          print('-'*80)
          model=MatrixFactorization(dataSource.M,dataSource.N,K,lr)
          simplex_models[model_name]=model
        # ======================================================================

        # move the simplex models to the GPU ===================================
        for model_name in simplex_models:
          model=simplex_models[model_name]
          model.to(device)
        # ======================================================================

        # train the simplex models and select B, G and W =======================
        train_models(simplex_models, users, items, ratings, epochs)
        l = [model for (_,model) in simplex_models.items()]
        l.sort()

        simplex_models['B']=l[0]
        simplex_models['G']=l[1]
        simplex_models['W']=l[2]
        # ======================================================================

        # create the auxiliary models ==========================================
        aux_models=build_aux_models(simplex_models)
        for model_name in aux_models:
          model=aux_models[model_name]
          model.to(device)
        # ======================================================================
        train_models(aux_models, users, items, ratings, epochs)

        models = {**simplex_models, **aux_models}
        start = False
      else:
        # not start

        if convergence:
          # convercence

          if number_of_batches_in_chunk == chunk_size:
            number_of_chunks+=1
            number_of_batches_in_chunk = 0   
          
          assert len(models)==1
        
          if pht.detected_change():
            # drift detected
            initialtimes.append(time.time())
            print("Drift detected!")
            pht.reset()

            # Create G
            K = K_range[int(K_range.size*offset)+randrange(int(K_range.size/gamma))]
            lr = round(lr_range[int(lr_range.size*offset)+randrange(int(lr_range.size/gamma))],6)
            simplex_models['G'] = MatrixFactorization(simplex_models['B'].M,simplex_models['B'].N,K,lr,simplex_models['B'])
            simplex_models['G'].to(device)

            # Create W
            K = K_range[int(K_range.size*offset)+randrange(int(K_range.size/gamma))]
            lr = round(lr_range[int(lr_range.size*offset)+randrange(int(lr_range.size/gamma))],6)
            simplex_models['W'] = MatrixFactorization(simplex_models['B'].M,simplex_models['B'].N,K,lr,simplex_models['B'])
            simplex_models['W'].to(device)

            # train the simplex models and select B, G and W ===================
            train_models(simplex_models, users, items, ratings, epochs)
            l = [model for (_,model) in simplex_models.items()]
            l.sort()

            simplex_models['B']=l[0]
            simplex_models['G']=l[1]
            simplex_models['W']=l[2]
            # ==================================================================

            # create the auxiliary models ======================================
            aux_models=build_aux_models(simplex_models)
            for model_name in aux_models:
              model=aux_models[model_name]
              model.to(device)
            # ==================================================================
            train_models(aux_models, users, items, ratings, epochs)

            models = {**simplex_models, **aux_models}
            convergence=False
          else:
            # no drift

            # train only the best model (B) ====================================
            train_models(models, users, items, ratings, epochs)

        else:
          # not convercence

          if number_of_batches_in_chunk == chunk_size:
            # number_of_batches_in_chunk == chunk_size

            # Run Nelder Mead procedure ========================================
            print('Running Nelder Mead procedure...')

            number_of_chunks+=1
            number_of_batches_in_chunk=0

            # Order B, G, W 
            l = [model for (_,model) in simplex_models.items()]
            l.sort()

            simplex_models['B']=l[0]
            simplex_models['G']=l[1]
            simplex_models['W']=l[2]
          
            if aux_models['R']<simplex_models['G']:
              if simplex_models['B']<aux_models['R']:
                simplex_models['W']=aux_models['R']
              else:
                if aux_models['E']<simplex_models['B']:
                  simplex_models['W']=aux_models['E']
                else:
                  simplex_models['W']=aux_models['R']
            else:
                if aux_models['R']<simplex_models['W']:
                  simplex_models['W']=aux_models['R']
                else:
                  if aux_models['C']<simplex_models['W']:
                    simplex_models['W']=aux_models['C']
                  else:
                    if aux_models['S']<simplex_models['W']:
                      simplex_models['W']=aux_models['S']
                    if aux_models['M']<simplex_models['G']:
                      simplex_models['G']=aux_models['M']

            print("Model B - K: ",simplex_models['B'].K," LR: ",simplex_models['B'].lr)
            print("Model G - K: ",simplex_models['G'].K," LR: ",simplex_models['G'].lr)
            print("Model W - K: ",simplex_models['W'].K," LR: ",simplex_models['W'].lr)

            m1 = np.array((simplex_models['B'].K ,simplex_models['B'].lr))
            m2 = np.array((simplex_models['G'].K ,simplex_models['G'].lr))
            m3 = np.array((simplex_models['W'].K ,simplex_models['W'].lr))

            dist1 = np.linalg.norm(m1-m2)
            dist2 = np.linalg.norm(m1-m3)
            dist3 = np.linalg.norm(m2-m3)

            # compute the max distance among the simplex models ================
            maxDist = 0;
            if dist1 >= maxDist:
              maxDist = dist1;
            if dist2 >= maxDist:
              maxDist = dist2;
            if dist3 >= maxDist:
              maxDist = dist3;
            # ==================================================================

            ndim = 2;
            rval = maxDist * math.sqrt((ndim / (2 * (ndim + 1))));

            print("Dist1: ",dist1," Dist2: ",dist2," Dist3: ",dist3, " Rval: ",rval)

            if rval<=convergencesphere:
              endtimes.append(time.time())
              print("Convergence!")
              end_time_convergence = time.time()
              count_examples_convergence = batch_size * chunk_size * number_of_chunks 
              convergence=True
              pht.reset()

              # delete all the models except the best one (B) ==================
              del simplex_models['G']
              del simplex_models['W']
              del aux_models
              models = {**simplex_models}
              # ================================================================
            else:
              # compute the new auxiliary models ===============================
              aux_models=build_aux_models(simplex_models)
              for model_name in aux_models:
                model=aux_models[model_name]
                model.to(device)
              models = {**simplex_models, **aux_models}
              # ================================================================

              scatter_list = plot_models(simplex_models,aux_models,K_range,lr_range, plot_lines=True,show=plot_training)
              plots.append(scatter_list)

          # train all the models (not start) ===================================
          train_models(models, users, items, ratings, epochs)
          # ====================================================================

      # updating statistics ====================================================
      total_loss+=simplex_models['B'].loss
      average_loss = total_loss/(number_of_batches)
      average_loss_values.append(average_loss)
      batch_loss_values.append(simplex_models['B'].loss)
      PHm_T.append(pht.add_element(simplex_models['B'].loss))
      window.append(simplex_models['B'].loss)

      if len(window)>window_size:
        window.pop(0)
        window_loss.append(mean(window))
      # ============================================================================

      if number_of_batches%(chunk_size)==0:
          print('number of chunks:',number_of_chunks)
          print('batch size:',batch_size)
          print('chunk_size:', chunk_size)
          print('epochs for each batch:',epochs)
          print('last batch:',number_of_batches,'/',dataSource.batches,sep='')
          print("last batch loss:", simplex_models['B'].loss)
          print('HP1: ', simplex_models['B'].K,' HP2: ', simplex_models['B'].lr)
          print("average_loss:", average_loss)
          print('-'*80)
      # if plot_training:
      #   clear_output(wait=True)
        
      # scatter_list = plot_models(simplex_models,aux_models,K_range,lr_range, plot_lines=True,show=plot_training)
      # plots.append(scatter_list)
      if stop_chunk is not None and number_of_chunks == stop_chunk:
        break

    end_time_runtime = time.time()-start_time_runtime
    count_examples_runtime = batch_size * chunk_size * number_of_chunks 
    average_loss = total_loss/number_of_batches
    
    results = {
        'average_loss_values':average_loss_values,
        'batch_loss_values':batch_loss_values,
        'PHm_T':PHm_T,
        'window_loss':window_loss,
        'end_time_runtime':end_time_runtime,
        'count_examples_runtime':count_examples_runtime,
        'average_loss':average_loss,
        'plots':plots
    }

    return results

# **SMAC**

In [ ]:
def process_smac_streaming(dataSource, pht_threshold, batch_size, chunk_size, window_size, debug=True):
  t0 = time.time()
  # Initialize configuration space ===============================================
  logging.basicConfig(level=logging.INFO)  
  cs = ConfigurationSpace()
  K = UniformIntegerHyperparameter("K", 2.0, 202.0)
  lr = UniformFloatHyperparameter("lr", 1e-5, 1e-1)
  cs.add_hyperparameters([K,lr])
  PHm_T = []
  pht=PageHinkleyJG(threshold=pht_threshold)
  # Set SMAC parameters ==========================================================
  scenario = Scenario({"run_obj": "quality", 
                      "runcount-limit": 20,
                      "cs": cs,
                      "deterministic": "true",
                      "limit_resources": False
                      }) 
  cost_per_batch = []
  batch_loss = []
  window_loss = []
  average_loss_values = [] 
  first=True
  

  # Function to train SMAC =======================================================
  def model_from_cfg(cfg):

      # Load K and lr ============================================================
      cfg = {k: cfg[k] for k in cfg if cfg[k]}
      # ==========================================================================
      
      nonlocal best_model
      nonlocal dataSource
      global epochs
      nonlocal cost_per_batch
      nonlocal batch_loss
      nonlocal window_loss
      nonlocal first
      nonlocal start_idx
      nonlocal end_idx
      nonlocal average_loss_values
      nonlocal PHm_T
      nonlocal pht
      PHm_T_aux = []
      pht_aux=PageHinkleyJG(threshold=pht_threshold)
      cost_per_batch_aux = []
      batch_loss_all_aux = []
      window_loss_aux = []
      average_loss_values_aux = []
      model = MatrixFactorization(dataSource.M,dataSource.N,**cfg)
      model.to(device)
      current_total_loss=0
      number_of_minibatches=0
      t0 = time.time()
      print(start_idx)
      print(end_idx)
      for u,i,r in dataSource.iter():
        if number_of_minibatches<start_idx or number_of_minibatches >= end_idx:
          number_of_minibatches +=1
          continue;
        users = torch.LongTensor(u).to(device)
        items = torch.LongTensor(i).to(device)
        ratings = torch.FloatTensor(r).to(device)
        number_of_minibatches +=1
        for epoch in range(epochs):
          model.optimizer.zero_grad()
          prediction = model(users, items)
          loss = loss_func(prediction, ratings)
          model.loss = loss.item()
          loss.backward()
          model.optimizer.step()

        current_total_loss += model.loss
        if first:
          cost_per_batch_aux.append(model.loss)
          batch_loss_all_aux.append(model.loss)
          PHm_T_aux.append(pht_aux.add_element(model.loss))
          average_loss = current_total_loss/(number_of_minibatches)
          average_loss_values_aux.append(average_loss)
          if (len(cost_per_batch_aux)>window_size):
            cost_per_batch_aux.pop(0)
            window_loss_aux.append(mean(cost_per_batch_aux))
      
      
      current_loss = current_total_loss/number_of_minibatches
      
      t1 = time.time()
      total = t1-t0
      
      if (best_model is None) and (first==True):
        cost_per_batch = []
        batch_loss = []
        window_loss = []
        average_loss_values = []
        PHm_T = []
        cost_per_batch.extend(cost_per_batch_aux)
        batch_loss.extend(batch_loss_all_aux)
        window_loss.extend(window_loss_aux)
        average_loss_values.extend(average_loss_values_aux)
        PHm_T.extend(PHm_T_aux)
        pht=pht_aux
        # Store loss and model ===================================================
        best_model = (total,number_of_minibatches,current_total_loss, model)
        # ========================================================================

        # Not first run ==========================================================
      elif best_model is None:
        best_model = (total,number_of_minibatches,current_total_loss, model)
        print('Model replaced with incumbent')

      else:
        # Check if new model is better than the previous one =====================
        # Store if better along with hyper-parameters ============================
        if current_total_loss < best_model[2]:
          if first:
            cost_per_batch = []
            batch_loss = []
            window_loss = []
            average_loss_values = []
            PHm_T = []
            cost_per_batch.extend(cost_per_batch_aux)
            batch_loss.extend(batch_loss_all_aux)
            window_loss.extend(window_loss_aux)
            average_loss_values.extend(average_loss_values_aux)
            PHm_T.extend(PHm_T_aux)
            pht=pht_aux
          best_model = (total, number_of_minibatches,current_total_loss, model)
          cs = ConfigurationSpace()
          K = UniformIntegerHyperparameter("K", 2.0, 202.0, default_value=best_model[3].K)
          lr = UniformFloatHyperparameter("lr", 1e-5, 1e-1, default_value=best_model[3].lr)
          cs.add_hyperparameters([K,lr])  
      # ==========================================================================

      return current_total_loss # Minimize

  best_model = None
  start_idx=0
  end_idx=chunk_size
  smac = SMAC4HPO(scenario=scenario, rng=np.random.RandomState(42), tae_runner=model_from_cfg)
  incumbent = smac.optimize()
  # Configuration found by SMAC ==================================================
  incumbent
  # Best model after training on first chunk =====================================
  best_model
  first=False
  
  current_total_loss = best_model[2]
  number_of_minibatches = chunk_size
  number_of_chunks = 1
  number_of_batches_in_chunk = 0
  
  # Use model parameters for testing =========================================
  model = best_model[3] 
  model.to(device)
  # Train model on new data ==================================================
  aux_count=0
  for u,i,r in dataSource.iter():
    if aux_count>=chunk_size:
      users = torch.LongTensor(u).to(device)
      items = torch.LongTensor(i).to(device)
      ratings = torch.FloatTensor(r).to(device)
      number_of_minibatches +=1
      number_of_batches_in_chunk+=1
      if number_of_batches_in_chunk == chunk_size:
        number_of_chunks+=1
        number_of_batches_in_chunk = 0
      for epoch in range(epochs):
        model.optimizer.zero_grad()
        prediction = model(users, items)
        loss = loss_func(prediction, ratings)
        model.loss = loss.item()
        loss.backward()
        model.optimizer.step()

      current_total_loss += model.loss
      cost_per_batch.append(model.loss)
      batch_loss.append(model.loss)  
      average_loss = current_total_loss/(number_of_minibatches)
      average_loss_values.append(average_loss)
      PHm_T.append(pht.add_element(model.loss))
      if (len(cost_per_batch)>window_size):
        cost_per_batch.pop(0)
        window_loss.append(mean(cost_per_batch))
      if pht.detected_change():
        print("**** Drift! ****")
        best_model = None
        pht.reset()
        start_idx = number_of_minibatches-chunk_size/2
        end_idx = number_of_minibatches+chunk_size/2
        cs = ConfigurationSpace()
        K = UniformIntegerHyperparameter("K", 2.0, 202.0)
        lr = UniformFloatHyperparameter("lr", 1e-5, 1e-1)
        cs.add_hyperparameters([K,lr])
        smac = SMAC4HPO(scenario=scenario,rng=np.random.RandomState(42),tae_runner=model_from_cfg)
        incumbent = smac.optimize()
        model = best_model[3] 
        model.to(device)
    aux_count+=1

  end_time_runtime = time.time()-t0
  average_loss = current_total_loss/(number_of_minibatches)
  count_examples_runtime = batch_size * chunk_size * number_of_chunks 
  results = {
        'average_loss_values':average_loss_values,
        'batch_loss_values':batch_loss,
        'PHm_T':PHm_T,
        'window_loss':window_loss,
        'end_time_runtime':end_time_runtime,
        'count_examples_runtime':count_examples_runtime,
        'average_loss':average_loss,
        'plots':None
  }

  return results

# **Matrix Factorization (hyperparameters computed by SMAC)**

In [ ]:
def process_default_streaming(dataSource, p_K, p_lr, pht_threshold, batch_size, chunk_size, window_size, debug=True):
  t0 = time.time()
  average_loss_values = []
  cost_per_batch = []
  batch_loss_values = []
  window_loss = []
  model = MatrixFactorization(dataSource.M,dataSource.N,K=p_K,lr=p_lr)
  model.to(device)
  total_loss=0
  number_of_batches=0
  number_of_chunks=0
  number_of_batches_in_chunk=0
  PHm_T = []
  pht=PageHinkleyJG(threshold=pht_threshold)
  
  for u,i,r in dataSource.iter():
    users = torch.LongTensor(u).to(device)
    items = torch.LongTensor(i).to(device)
    ratings = torch.FloatTensor(r).to(device)
    number_of_batches +=1
    number_of_batches_in_chunk+=1
    if number_of_batches_in_chunk == chunk_size:
      number_of_chunks+=1
      number_of_batches_in_chunk = 0
    for epoch in range(epochs):
      model.optimizer.zero_grad()
      prediction = model(users, items)
      loss = loss_func(prediction, ratings)
      model.loss = loss.item()
      loss.backward()
      model.optimizer.step()

    total_loss += model.loss
    PHm_T.append(pht.add_element(model.loss))
    cost_per_batch.append(model.loss)
    batch_loss_values.append(model.loss)
    average_loss = total_loss/(number_of_batches)
    average_loss_values.append(average_loss)
    if (len(cost_per_batch)>window_size):
      cost_per_batch.pop(0)
      window_loss.append(mean(cost_per_batch))

  end_time_runtime = time.time() - t0
  count_examples_runtime = batch_size * chunk_size * number_of_chunks
  average_loss = total_loss/number_of_batches
  results = {
    'average_loss_values':average_loss_values,
    'batch_loss_values':batch_loss_values,
    'PHm_T':PHm_T,
    'window_loss':window_loss,
    'end_time_runtime':end_time_runtime,
    'count_examples_runtime':count_examples_runtime,
    'average_loss':average_loss,
    'plots':None
  }
  return results

# **MovieLens**

##**Experiments**

In [ ]:
ml_batch_size = 200 
ml_chunk_size = 10 
ml_window_size = 50 

In [ ]:
if dataset==ML_25M:
  dataLoader=DataLoader(url='http://files.grouplens.org/datasets/movielens/ml-25m.zip', 
                        filename='/ml-25m/ratings.csv',
                        archive_name='dataset_ml_25m.zip',
                        sep=',',
                        skiprows=1,
                        batch_size = ml_batch_size,
                        force_download=False)

elif dataset==ML_1M:
  dataLoader=DataLoader(batch_size = ml_batch_size)

dataLoader.df.head(2000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



Number of Users: 6040
Number of Items: 3952
Samples: 1000209


,User,Item,Rating,Timestamp
1000138,6039,857,4,956703932
1000153,6039,2383,4,956703954
999873,6039,592,5,956703954
1000007,6039,1960,4,956703977
1000192,6039,2018,5,956703977
...,...,...,...,...
997764,6024,2395,4,956731134
997636,6024,16,5,956731134
997727,6024,2691,5,956731134
997701,6024,2290,4,956731168


In [ ]:
nelder_mead_movie_lens = process_streaming(dataLoader,0.5,batch_size=ml_batch_size, chunk_size=ml_chunk_size, window_size=ml_window_size)

Initializing model B
--------------------------------------------------------------------------------
Initializing model G
--------------------------------------------------------------------------------
Initializing model W
--------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning:

torch.lstsq is deprecated in favor of torch.linalg.lstsq and will be removed in a future PyTorch release.
torch.linalg.lstsq has reversed arguments and does not return the QR decomposition in the returned tuple (although it returns other information about the problem).
To get the qr decomposition consider using torch.linalg.qr.
The returned solution in torch.lstsq stored the residuals of the solution in the last m - n columns of the returned value whenever m > n. In torch.linalg.lstsq, the residuals in the field 'residuals' of the returned named tuple.
The unpacking of the solution, as in
X, _ = torch.lstsq(B, A).solution[:A.size(1)]
should be replaced with
X = torch.linalg.lstsq(A, B).solution (Triggered internally at  /pytorch/aten/src/ATen/LegacyTHFunctionsCPU.cpp:389.)



Streaming output truncated to the last 5000 lines.
Model G - K:  156  LR:  0.01
Model W - K:  162  LR:  0.01
Dist1:  12.0  Dist2:  18.0  Dist3:  6.0  Rval:  10.392304845413264
number of chunks: 74
batch size: 200
chunk_size: 10
epochs for each batch: 20
last batch:740/5002
last batch loss: 0.7168422937393188
HP1:  144  HP2:  0.01
average_loss: 0.791387509111617
--------------------------------------------------------------------------------
Running Nelder Mead procedure...
Model B - K:  144  LR:  0.01
Model G - K:  162  LR:  0.01
Model W - K:  138  LR:  0.01
Dist1:  18.0  Dist2:  6.0  Dist3:  24.0  Rval:  13.856406460551018
number of chunks: 75
batch size: 200
chunk_size: 10
epochs for each batch: 20
last batch:750/5002
last batch loss: 0.5224336385726929
HP1:  144  HP2:  0.01
average_loss: 0.7879531470735868
--------------------------------------------------------------------------------
Running Nelder Mead procedure...
Model B - K:  144  LR:  0.01
Model G - K:  138  LR:  0.01
Model W

In [ ]:
smac_movie_lens=process_smac_streaming(dataLoader,0.5,batch_size=ml_batch_size, chunk_size=ml_chunk_size, window_size=ml_window_size)

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-06-29_17:19:53_875448
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 5 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


0
10


INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 7.3883


0
10


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 2.304459 sec, available: 0.000010 sec)


...

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 1.071635 sec, available: 0.000010 sec)


509.0
519.0


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 1.043280 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 4
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 20 / 20.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 20 / 20.0
INFO:smac.stats.stats.Stats:#Configurations: 20
INFO:smac.stats.stats.Stats:Used wallclock time: 27.34 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 21.49 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  K, Value: 33
  lr, Value: 0.08448136498155767

INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Estimated cost of incumbent: 3.842287


**** Drift! ****


OSError: ignored

In [ ]:
default_movie_lens = process_default_streaming(dataLoader,p_K=35,p_lr=0.033720415892090044, pht_threshold=1.0,batch_size=ml_batch_size, chunk_size=ml_chunk_size, window_size=ml_window_size)

##**Plots**

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_movie_lens['average_loss_values'],label='Nelder-Mead')
plt.plot(smac_movie_lens['average_loss_values'],label='SMAC Baseline')
plt.plot(default_movie_lens['average_loss_values'],label='Static Baseline')
plt.yscale('log')

plt.legend()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_movie_lens['window_loss'],label='Nelder-Mead')
plt.plot(smac_movie_lens['window_loss'],label='SMAC Baseline')
plt.plot(default_movie_lens['window_loss'],label='Static Baseline')
plt.yscale('log')

plt.legend()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_movie_lens['PHm_T'],label='Nelder-Mead')
plt.plot(smac_movie_lens['PHm_T'],label='SMAC Baseline')
plt.plot(default_movie_lens['PHm_T'],label='Static Baseline')


plt.legend()

# **Synthetic Data (a)**

##**Experiments**

In [ ]:
dg_batch_size = 2000
dg_chunk_size = 10 
dg_window_size = 10

In [ ]:
dataGenerator = DataGenerator_1(N=10000, 
                                M=2000, 
                                K=100, 
                                batches=8000, 
                                batch_size=dg_batch_size, 
                                α=1.1, 
                                T=2000, 
                                gaussian_sampling=False,
                                unassign_old_features=False)
dataGenerator.set_deterministic(True)

In [ ]:
nelder_mead_datagenerator = process_streaming(dataGenerator,0.05,batch_size=dg_batch_size, chunk_size=dg_chunk_size, window_size=dg_window_size)

In [ ]:
smac_datagenerator=process_smac_streaming(dataGenerator,1.0,batch_size=dg_batch_size, chunk_size=dg_chunk_size, window_size=dg_window_size)

In [ ]:
default_datagenerator = process_default_streaming(dataGenerator,p_K=76,p_lr=0.03701365688098193, pht_threshold=1.0,batch_size=dg_batch_size, chunk_size=dg_chunk_size, window_size=dg_window_size)

##**Plots**

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_datagenerator['average_loss_values'],label='Nelder-Mead')
plt.plot(smac_datagenerator['average_loss_values'],label='SMAC Baseline')
plt.plot(default_datagenerator['average_loss_values'],label='Static Baseline')
plt.yscale('log')
plt.legend()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_datagenerator['window_loss'],label='Nelder-Mead')
plt.plot(smac_datagenerator['window_loss'],label='SMAC Baseline')
plt.plot(default_datagenerator['window_loss'],label='Static Baseline')
plt.yscale('log')

plt.legend()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_datagenerator['PHm_T'],label='Nelder-Mead')
plt.plot(smac_datagenerator['PHm_T'],label='SMAC Baseline')
plt.plot(default_datagenerator['PHm_T'],label='Static Baseline')

plt.legend()

# **Synthetic Data (b)**

##**Experiments**

In [ ]:
dg_b_batch_size = 2000
dg_b_chunk_size = 10 
dg_b_window_size = 10

In [ ]:
dataGenerator_b = DataGenerator_1(N=10000, 
                                M=2000, 
                                K=100, 
                                batches=8000, 
                                batch_size=dg_batch_size, 
                                α=1.1, 
                                p=0.9, 
                                p_d=0.5,
                                p_r=0.5,
                                p_a=0.5,
                                gaussian_sampling=False,
                                variance=1,
                                T=2000, 
                                unassign_old_features=False)
dataGenerator_b.set_deterministic(True)

In [ ]:
nelder_mead_datagenerator_b = process_streaming(dataGenerator_b,0.05,batch_size=dg_b_batch_size, chunk_size=dg_b_chunk_size, window_size=dg_b_window_size)

In [ ]:
smac_datagenerator_b=process_smac_streaming(dataGenerator_b,1.0,batch_size=dg_batch_size, chunk_size=dg_chunk_size, window_size=dg_b_window_size)

In [ ]:
default_datagenerator_b = process_default_streaming(dataGenerator_b,p_K=101,p_lr=0.03802782813491344, pht_threshold=1.0,batch_size=dg_batch_size, chunk_size=dg_chunk_size, window_size=dg_b_window_size)

##**Plots**

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_datagenerator_b['average_loss_values'],label='Nelder-Mead')
plt.plot(smac_datagenerator_b['average_loss_values'],label='SMAC Baseline')
plt.plot(default_datagenerator_b['average_loss_values'],label='Static Baseline')
plt.yscale('log')
plt.legend()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_datagenerator_b['window_loss'],label='Nelder-Mead')
plt.plot(smac_datagenerator_b['window_loss'],label='SMAC Baseline')
plt.plot(default_datagenerator_b['window_loss'],label='Static Baseline')
plt.yscale('log')

plt.legend()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 3), dpi=300)
plt.plot(nelder_mead_datagenerator_b['PHm_T'],label='Nelder-Mead')
plt.plot(smac_datagenerator_b['PHm_T'],label='SMAC Baseline')
plt.plot(default_datagenerator_b['PHm_T'],label='Static Baseline')

plt.legend()